In [2]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp,shapiro,levene,ttest_ind, \
    mannwhitneyu,pearsonr,spearmanr,kendalltau,f_oneway,kruskal
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
# Görev 1: Veriyi Hazırlama ve Analiz Etme

In [24]:
# Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz.
# Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

dfc = pd.read_excel("ab_testing.xlsx", sheet_name= "Control Group")

dft = pd.read_excel("ab_testing.xlsx", sheet_name= "Test Group")

In [26]:
# Adım 2: Kontrol ve test grubu verilerini analiz ediniz.
dft.head()
dfc.head()

dft.describe().T
dfc.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,101711.449068,20302.157862,45475.942965,85726.690349,99790.701078,115212.816543,147539.336329
Click,40.0,5100.657373,1329.985498,2189.753157,4124.304129,5001.220602,5923.803596,7959.125069
Purchase,40.0,550.894059,134.108201,267.028943,470.095533,531.206307,637.957088,801.795020
Earning,40.0,1908.568300,302.917783,1253.989525,1685.847205,1975.160522,2119.802784,2497.295218


In [29]:
#Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

dfc["Bidding"] = "Maximum_Bidding"

dfc.groupby("Bidding").agg({"Purchase": "mean"})

dft["Bidding"] = "Average_Bidding"

dft.groupby("Bidding").agg({"Purchase": "mean"})

df = pd.concat([dfc, dft])

df.head()

,Impression,Click,Purchase,Earning,Bidding
0,82529.459271,6090.077317,665.211255,2311.277143,Maximum_Bidding
1,98050.451926,3382.861786,315.084895,1742.806855,Maximum_Bidding
2,82696.023549,4167.965750,458.083738,1797.827447,Maximum_Bidding
3,109914.400398,4910.882240,487.090773,1696.229178,Maximum_Bidding
4,108457.762630,5987.655811,441.034050,1543.720179,Maximum_Bidding


In [ ]:
# Görev 2: A/B Testinin Hipotezinin Tanımlanması

In [ ]:
# Adım 1: Hipotezi tanımlayınız.
# H0 : M1 = M2 
# H1 : M1!= M2 

# H0 : M1 = M2  # Average bidding ve Maksimum Bidding purchase ortalamaları Arasında İst. Olarak An. Fark yoktur
# H1 : M1!= M2  # ..... vardır

In [30]:
# Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

df.groupby("Bidding").agg({"Purchase": "mean"})

,Purchase
Bidding,
Average_Bidding,582.106097
Maximum_Bidding,550.894059


In [ ]:
# Görev 3: Hipotez Testinin Gerçekleştirilmesi

In [31]:
# Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.

# Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni
# üzerinden ayrı ayrı test ediniz.

# Normallik Varsayımı :
# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1: Normal dağılım varsayımı sağlanmamaktadır.
# p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
# Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız.

test_stat, pvalue = shapiro(df.loc[df["Bidding"] == "Maximum_Bidding", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [32]:
test_stat, pvalue = shapiro(df.loc[df["Bidding"] == "Average_Bidding", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [33]:
# Varyans Homojenliği :
# H0: Varyanslar homojendir.
# H1: Varyanslar homojen Değildir.
# p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
# Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz.
# Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız.

test_stat, pvalue = levene(df.loc[df["Bidding"] == "Maximum_Bidding", "Purchase"],
                           df.loc[df["Bidding"] == "Average_Bidding", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [34]:
# Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

# Her iki varsayım da sağlanıyor. Yani bağımsız iki örneklem t testi yapacağız. (parametrik test)

In [35]:
test_stat, pvalue = ttest_ind(df.loc[df["Bidding"] == "Maximum_Bidding", "Purchase"],
                           df.loc[df["Bidding"] == "Average_Bidding", "Purchase"],
                            equal_var=True)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

## p-value değeri 0.3493 geldi. Yani H0 hipotezini reddedemiyoruz.
# Yani her iki yöntem arasında anlamlı bir fark var diyemeyiz. H0: M1 = M2 reddedilemez

Test Stat = -0.9416, p-value = 0.3493


In [ ]:
# Görev 4: Sonuçların Analizi

# Varsayımlar sağlandığı için bağımsız iki örneklem t testi yaptık.

# Elde ettiğimiz sonuca göre average bidding ile maximum bidding arasında
# purchase ortalamaları açısından anlamlı bir fark olmadığını gördük.

# Müşteriden veri sayısını arttırmasını isteyebiliriz.
# 3 ay daha veri toplamasını ve tekrar hipotez testi yapılmasını isteyebiliriz.

# Mevcut sonuçlara göre yeni teklif türü olan average biddingi satın almamasını tavsiye ederiz.